In [1]:
import cv2
import numpy as np
import dlib
import os
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt
from skimage.feature import hog
from skimage import exposure

In [2]:
dir_train = r'.\cropped_dataset\train'
dir_test = r'.\cropped_dataset\test'
dir_model = r'.\model'


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(dir_model + "\\" + "shape_predictor_68_face_landmarks.dat")

In [3]:
def edge_segmentation(image):
    edges = cv2.Canny(image, 100, 200)
    return edges

# Feature extraction (dummy example)

def extract_features(image):
    # Resize the image to 128x128
    image_resized = cv2.resize(image, (256, 256))
    # Compute HOG descriptors
    features, hog_image = hog(image_resized, orientations=9, pixels_per_cell=(8, 8),
                              cells_per_block=(4, 4), block_norm="L2", visualize=True)
    return features, hog_image

def apply_kmeans(image, n_clusters=4):
    pixels = image.reshape(-1, 3)
    kmeans = KMeans(n_clusters=n_clusters, n_init="auto")
    kmeans.fit(pixels)
    
    segmented_image = kmeans.cluster_centers_[kmeans.labels_]
    segmented_image = segmented_image.reshape(image.shape).astype('uint8')

    return segmented_image

## VGG16 - VGGFace

### Import libraries


In [ ]:
!pip install tensorflow

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import cv2
import random
import pickle
import itertools

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras import optimizers
from tensorflow.keras import utils
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.vgg16 import VGG16
from sklearn.metrics import confusion_matrix
from keras.utils import to_categorical 
%matplotlib inline


### Define Functions

In [1]:
def plot_results(mod_history, metric, epochs):
      
      # Check out our train loss and test loss over epochs.
      train_metric = mod_history.history[metric]
      val = 'val_' + metric
      test_metric = mod_history.history[val]

      # Set figure size.
      plt.figure(figsize=(12, 8))

      # Generate line plot of training, testing loss over epochs.
      plt.plot(train_metric, label=f'Training {metric}', color='#185fad')
      plt.plot(test_metric, label=f'Testing {metric}', color='orange')

      # Set title
      plt.title(f'Training and Testing {metric} by Epoch', fontsize = 25)
      plt.xlabel('Epoch', fontsize = 18)
      plt.ylabel('Categorical Crossentropy', fontsize = 18)
      plt.xticks(range(0,epochs,5), range(0,epochs,5))
      plt.legend(fontsize = 18);

In [ ]:
def make_predictions(mod_name, steps=20):
    preds = mod_name.predict(X_test,steps=steps)
    preds = preds.argmax(axis=-1)

    y_test_labels = np.argmax(y_test, axis=-1)

    cm = confusion_matrix(y_test_labels,preds)

    plot_confusion_matrix(cm, cm_plot_labels, normalize=True,
                          title='Face Shape Normalized')

    plt.show()



In [ ]:
cm_plot_labels = ['Heart','Oblong','Oval','Round', 'Square']

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.figure(figsize=(16,8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [ ]:
def plot_summary_results(mod_name, mod_history, epochs):
    plot_results(mod_history, 'loss',epochs)
    plot_results(mod_history, 'accuracy', epochs)
    make_predictions(mod_name)

### Load Data Files

In [ ]:
def feature_engineering(path):
    labels = []
    images = []
    shape_types = ['Heart', 'Oblong', 'Oval', 'Round', 'Square']

    for shape in shape_types:
        print(f'Loading {shape} images...')
        shape_dir = os.path.join(path, shape)
        if not os.path.exists(shape_dir):
            print(f"Directory {shape_dir} does not exist.")
            continue
        for imgName in os.listdir(shape_dir):
            img_path = os.path.join(shape_dir, imgName)
            if not os.path.isfile(img_path):
                print(f"File {img_path} does not exist.")
                continue
            img = cv2.imread(img_path)
            if img is None:
                print(f"Image {img_path} could not be read.")
                continue
            # Resize image to desired size (224, 224)
            img = cv2.resize(img, (224, 224))
            images.append(img)
            labels.append(shape_types.index(shape))

    labels = to_categorical(labels, num_classes=len(shape_types))
    images = np.array(images)

    return images, labels

In [ ]:
dir_train = "/kaggle/input/face-shape-preprocessed/cropped_dataset/train"
dir_test = "/kaggle/input/face-shape-preprocessed/cropped_dataset/test"
X_train, y_train = feature_engineering(dir_train)
X_test, y_test = feature_engineering(dir_test)

In [ ]:
# Normalize pixel values to be between 0 and 1
X_train = X_train / 255.0
X_test = X_test / 255.0

In [ ]:
print("Data Summary")
print("--------------------")
print(f"X_train shape {X_train.shape}")
print(f"y_train shape {y_train.shape}")
print("--------------------")
print(f"X_test shape {X_test.shape}")
print(f"y_test shape {y_test.shape}")

## Model TL: Transfer Learning from VGG16 with weights from VGG Face

In [ ]:
# Path for VGGFace weights

path_vggface = '/kaggle/input/vggface16/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'

In [ ]:
# Loading VGG16 as base model
base_model = VGG16(input_shape=(224, 224, 3),  # same as our input
                   include_top=False,  # exclude the last layer
                   weights=path_vggface)  # use VGGFace Weights

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
model_t1 = Sequential() # 

In [ ]:
# Setting up the layer
from tensorflow.keras import regularizers
x = base_model.output
#x = layers.GlobalAveragePooling2D()(x)

x = layers.Flatten()(x)      # flatten the output of the Base Model

# x = layers.Dense(512, activation='relu')(x)  # add 1 fully connected layer, try with 512 neurons first

x = layers.Dense(512, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)                   # add a dropout layer set 0.5 or 50% of the inputs unit to 0 at each update during training 

# x = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.0001))(x)
# x = layers.BatchNormalization()(x)
# x = layers.Dropout(0.5)(x)                   # add a dropout layer set 0.5 or 50% of the inputs unit to 0 at each update during training 

x = layers.Dense(5, activation='softmax')(x)  # add final layer

model_t1 = tf.keras.models.Model(base_model.input, x) # create the Model

# Compile and Fit the model

model_t1.compile(loss='categorical_crossentropy', # categorical_crossentropy loss used for multiclass classification 
                 optimizer='adam',
                 metrics=['accuracy'])

model_t1.summary()

In [ ]:
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)

In [ ]:
datagen.fit(X_train)

In [ ]:
# history_t1 = model_t1.fit(datagen.flow(X_train, y_train, batch_size=32), 
#                           steps_per_epoch=len(X_train)/32, epochs=80, 
#                           validation_data=(X_test, y_test))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
history_t1 = model_t1.fit(datagen.flow(X_train, y_train, batch_size=32), 
                          steps_per_epoch=len(X_train)/32, epochs=50, 
                          validation_data=(X_test, y_test),
                          callbacks = [lr_scheduler])

In [ ]:
model_path = '/kaggle/working/saved_models/'
tf.keras.models.save_model(
    model_t1, filepath=model_path, overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None)

In [ ]:
filename = model_path + 'vgg16-face-1'   # change the filename for new iterations
model_t1.save(filename)

In [ ]:
loaded_model = tf.keras.models.load_model(filename)
mod_t1_predict = np.argmax(model_t1.predict(X_test), axis=1) 
loaded_t1_predict = np.argmax(loaded_model.predict(X_test), axis=1)

# Check the difference

print(f'Difference in predictions: Saved model vs. original model is {np.sum(loaded_t1_predict - mod_t1_predict)}\nModel was correctly saved.')

In [ ]:
plot_summary_results(model_t1, history_t1, 50)